<a href="https://colab.research.google.com/github/SnehaVPujari007/Sentimental-Analysis/blob/main/Sentimental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN , Dense  # Corrected the typo from 'Emedding' to 'Embedding'
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

In [2]:
## Load the imbd dataset

max_features = 10000  ## Vocalbulary size
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
print(f'X_train data shape: {X_train.shape} , y_train labels shape : {y_train.shape} ' )
print(f'X_test data shape: {X_test.shape} , y_test labels shape : {y_test.shape}')

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
X_train data shape: (25000,) , y_train labels shape : (25000,) 
X_test data shape: (25000,) , y_test labels shape : (25000,)


In [3]:
X_train[0] , y_train[0]

([1,
  14,
  22,
  16,
  43,
  530,
  973,
  1622,
  1385,
  65,
  458,
  4468,
  66,
  3941,
  4,
  173,
  36,
  256,
  5,
  25,
  100,
  43,
  838,
  112,
  50,
  670,
  2,
  9,
  35,
  480,
  284,
  5,
  150,
  4,
  172,
  112,
  167,
  2,
  336,
  385,
  39,
  4,
  172,
  4536,
  1111,
  17,
  546,
  38,
  13,
  447,
  4,
  192,
  50,
  16,
  6,
  147,
  2025,
  19,
  14,
  22,
  4,
  1920,
  4613,
  469,
  4,
  22,
  71,
  87,
  12,
  16,
  43,
  530,
  38,
  76,
  15,
  13,
  1247,
  4,
  22,
  17,
  515,
  17,
  12,
  16,
  626,
  18,
  2,
  5,
  62,
  386,
  12,
  8,
  316,
  8,
  106,
  5,
  4,
  2223,
  5244,
  16,
  480,
  66,
  3785,
  33,
  4,
  130,
  12,
  16,
  38,
  619,
  5,
  25,
  124,
  51,
  36,
  135,
  48,
  25,
  1415,
  33,
  6,
  22,
  12,
  215,
  28,
  77,
  52,
  5,
  14,
  407,
  16,
  82,
  2,
  8,
  4,
  107,
  117,
  5952,
  15,
  256,
  4,
  2,
  7,
  3766,
  5,
  723,
  36,
  71,
  43,
  530,
  476,
  26,
  400,
  317,
  46,
  7,
  4,
  2,
  1029,
  

In [4]:
##  Inspect a sample  review and its label

sample_review = X_train[0]
sample_label = y_train[0]

print("Sample Review (as integers):", sample_review)
print("Sample Label:", sample_label)

Sample Review (as integers): [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
Sample Label: 1


In [7]:
## Mapping of words index to words (for understanding)

word_index = imdb.get_word_index()
#word_index

In [8]:
reverse_word_index = {value: key for (key, value) in word_index.items()}
reverse_word_index

{34701: 'fawn',
 52006: 'tsukino',
 52007: 'nunnery',
 16816: 'sonja',
 63951: 'vani',
 1408: 'woods',
 16115: 'spiders',
 2345: 'hanging',
 2289: 'woody',
 52008: 'trawling',
 52009: "hold's",
 11307: 'comically',
 40830: 'localized',
 30568: 'disobeying',
 52010: "'royale",
 40831: "harpo's",
 52011: 'canet',
 19313: 'aileen',
 52012: 'acurately',
 52013: "diplomat's",
 25242: 'rickman',
 6746: 'arranged',
 52014: 'rumbustious',
 52015: 'familiarness',
 52016: "spider'",
 68804: 'hahahah',
 52017: "wood'",
 40833: 'transvestism',
 34702: "hangin'",
 2338: 'bringing',
 40834: 'seamier',
 34703: 'wooded',
 52018: 'bravora',
 16817: 'grueling',
 1636: 'wooden',
 16818: 'wednesday',
 52019: "'prix",
 34704: 'altagracia',
 52020: 'circuitry',
 11585: 'crotch',
 57766: 'busybody',
 52021: "tart'n'tangy",
 14129: 'burgade',
 52023: 'thrace',
 11038: "tom's",
 52025: 'snuggles',
 29114: 'francesco',
 52027: 'complainers',
 52125: 'templarios',
 40835: '272',
 52028: '273',
 52130: 'zaniacs',

In [9]:
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in sample_review])
decoded_review

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [10]:
from tensorflow.keras.preprocessing import sequence
max_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_length)

In [11]:
X_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

In [12]:
## Trining RNN

model = Sequential()
model.add(Embedding(max_features, 128, input_length = max_length))
model.add(SimpleRNN(128,activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [14]:
model.build(input_shape=(None, max_length))

In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 500, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,313,025 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
early_stop

In [19]:
## Train the model with early stopping
history =model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stop])

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 164s 260ms/step - accuracy: 0.5406 - loss: 149727.4219 - val_accuracy: 0.6152 - val_loss: 0.6492
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 198s 253ms/step - accuracy: 0.6808 - loss: 0.6201 - val_accuracy: 0.6640 - val_loss: 0.6073
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 161s 257ms/step - accuracy: 0.7507 - loss: 0.5462 - val_accuracy: 0.6776 - val_loss: 0.5963
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 201s 255ms/step - accuracy: 0.7894 - loss: 0.4633 - val_accuracy: 0.6860 - val_loss: 0.5981
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 199s 251ms/step - accuracy: 0.8203 - loss: 0.4040 - val_accuracy: 0.6906 - val_loss: 0.6220
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 198s 245ms/step - accuracy: 0.8514 - loss: 0.3481 - val_accuracy: 0.6922 - val_loss: 0.6385
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 203s 247ms/step - accuracy: 0.8671 - loss: 0.3143 - val_accuracy: 0.7092 - val_loss: 0.7335
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 205s 252ms/step - accuracy: 0.8

In [20]:
## Save model

model.save('rnn_model.h5')

In [22]:
import tensorflow as tf
from tensorflow.keras.models import load_model
model = load_model('rnn_model.h5')

In [23]:
model.get_weights()

[array([[-0.2793993 ,  0.20795272,  0.2197751 , ..., -0.14961627,
         -0.03824813, -0.3076827 ],
        [ 0.03653201,  0.00667821,  0.01716251, ..., -0.03677782,
         -0.01175343, -0.02729737],
        [-0.02367764, -0.00310537,  0.03864015, ...,  0.03397191,
          0.02640637,  0.0113851 ],
        ...,
        [ 0.01533801, -0.04793918,  0.01456404, ..., -0.05561702,
          0.08225893,  0.03184418],
        [-0.06966262,  0.07396264, -0.06210661, ...,  0.0493089 ,
         -0.08265792, -0.04679696],
        [ 0.01655309, -0.04007336,  0.02750126, ..., -0.04560243,
          0.08518682,  0.04052195]], dtype=float32),
 array([[ 0.00065804, -0.09807539, -0.10767057, ...,  0.07198753,
         -0.0839412 ,  0.15393308],
        [-0.07560026,  0.1696779 ,  0.02728922, ..., -0.11737794,
         -0.07979532,  0.13643293],
        [ 0.02308661, -0.12109881,  0.0986352 , ...,  0.11703398,
         -0.00939899, -0.10553034],
        ...,
        [ 0.02252338,  0.01819568,  0.0

In [24]:
def decoded_review(encoded_review):
  return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

def preprocess_text(text):
  words = text.lower().split()
  encoded_review = [word_index.get(word, 0) for word in words]
  padded_review = sequence.pad_sequences([encoded_review], maxlen=max_length)
  return padded_review


In [26]:
## Prediction Function
def predict_sentence(review):
  preprocess_text(review)
  prediction = model.predict(preprocess_text(review))
  sentiment = "Positive" if prediction[0][0] > 0.5 else "Negative"
  return sentiment,prediction[0][0]


In [27]:
example_review = "This move was fanatastic! The acting was great and the plot was thrilling"
sentiment,score = predict_sentence(example_review)
print(f"Review: {example_review}")
print(f"Sentiment: {sentiment}")
print(f"Score: {score} ")




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
Review: This move was fanatastic! The acting was great and the plot was thrilling
Sentiment: Negative
Score: 0.2724655270576477 
